<a href="https://colab.research.google.com/github/uteyechea/crime-prediction-using-artificial-intelligence/blob/master/temporal_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc

import pandas as pd
from scipy import stats

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path='/content/drive/My Drive/Colab Notebooks/crime_prediction'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Get a window of length t-n, where t is the end datetime and n is the length of the temporal window in a data series.

In [2]:
file_path=os.path.join(path,'data','theft.csv')
file=pd.read_csv(file_path,sep=',',parse_dates=['Date'],index_col='Date')

In [3]:
file

,zone0,zone1,zone2,zone3,zone4,zone5
Date,,,,,,
2001-01-01,0.0,16.0,33.0,19.0,0.0,47.0
2001-01-02,0.0,0.0,4.0,2.0,0.0,1.0
2001-01-03,0.0,0.0,0.0,0.0,0.0,1.0
2001-01-05,0.0,1.0,0.0,0.0,0.0,0.0
2001-01-06,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2020-07-04,0.0,18.0,17.0,26.0,0.0,21.0
2020-07-05,0.0,11.0,22.0,22.0,0.0,10.0
2020-07-06,0.0,22.0,27.0,34.0,0.0,21.0


In [175]:
def get_window(dataframe,t,n):
  #if type(t) == int elif type(t)==str then use .loc, instead of iloc. For the time being, we will just assume t and n are integers 
  return dataframe.iloc[t-n:t,:]

def get_correlation(dataframe1,dataframe2):
  assert dataframe1.shape==dataframe2.shape,'Dataframes must have the same shape'
  ro=dataframe1.corrwith(dataframe2.set_index(dataframe1.index),axis=0) #Force alignment. Make sure size is the same for both dataframes
  return ro.mean() #For the time being we will stop with a general correlation among all zones

def get_correlated_endof_sequence_timestamp(dataframe,t,n=10,min_correlation=0.5):
  endof_sequence_timestamp=[]
  dataframe2=get_window(dataframe,t,n)
  for epoch in reversed(range(2*n,t)):
    dataframe1=get_window(dataframe,epoch-n,n)
    correlation=get_correlation(dataframe1,dataframe2)
    if correlation >= min_correlation:
      #record datetime value. We will use this datetime value to generate all sequences that will go as input to the RNN
      #print(dataframe.index[epoch]) 
      endof_sequence_timestamp.append(dataframe.index[epoch])
  return endof_sequence_timestamp

def get_correlated_sequence_centeredat_timestamp(dataframe,endof_sequence_timestamp,n):
  input={}
  output={}
  for timestamp in endof_sequence_timestamp:
    input[timestamp]=file.loc[pd.date_range(start=timestamp,periods=n,freq='-1D')]
    output[timestamp]=file.loc[pd.date_range(start=timestamp,periods=n,freq='1D')]
  return input,output

def get_IO_series(dataframe,periods,min_correlation=0.5):
  endof_sequence_timestamp=get_correlated_endof_sequence_timestamp(dataframe,len(dataframe),periods)
  input,output=get_correlated_sequence_centeredat_timestamp(dataframe,endof_sequence_timestamp,periods)
  return input,output  

In [176]:
input,output=get_IO_series(file,periods=10)

It would be ideal to save a list of dates for each zone where the correlation is high. Nevertheless, we will demote this as further work. 

In [177]:
output

{Timestamp('2002-10-14 00:00:00'):             zone0  zone1  zone2  zone3  zone4  zone5
 2002-10-14    0.0   56.0   60.0   70.0    0.0   56.0
 2002-10-15    0.0   59.0   58.0   90.0    0.0   64.0
 2002-10-16    0.0   34.0   56.0   85.0    0.0   59.0
 2002-10-17    0.0   56.0   49.0   82.0    0.0   59.0
 2002-10-18    0.0   46.0   66.0   93.0    0.0   75.0
 2002-10-19    0.0   37.0   56.0  104.0    0.0   43.0
 2002-10-20    0.0   38.0   48.0   63.0    0.0   43.0
 2002-10-21    0.0   45.0   42.0   73.0    0.0   53.0
 2002-10-22    0.0   45.0   48.0   78.0    0.0   57.0
 2002-10-23    0.0   51.0   66.0   59.0    0.0   54.0,
 Timestamp('2002-11-24 00:00:00'):             zone0  zone1  zone2  zone3  zone4  zone5
 2002-11-24    0.0   33.0   47.0   65.0    0.0   44.0
 2002-11-25    0.0   52.0   55.0   71.0    0.0   40.0
 2002-11-26    0.0   32.0   60.0   77.0    0.0   42.0
 2002-11-27    0.0   56.0   59.0   79.0    0.0   44.0
 2002-11-28    0.0   28.0   20.0   38.0    0.0   41.0
 2002-11-29  

In [178]:
input

{Timestamp('2002-10-14 00:00:00'):             zone0  zone1  zone2  zone3  zone4  zone5
 2002-10-14    0.0   56.0   60.0   70.0    0.0   56.0
 2002-10-13    0.0   38.0   43.0   53.0    0.0   38.0
 2002-10-12    0.0   55.0   58.0   84.0    0.0   48.0
 2002-10-11    0.0   46.0   84.0   87.0    0.0   50.0
 2002-10-10    0.0   47.0   49.0   89.0    0.0   52.0
 2002-10-09    0.0   47.0   64.0   77.0    0.0   62.0
 2002-10-08    0.0   53.0   71.0   87.0    0.0   50.0
 2002-10-07    0.0   40.0   55.0   94.0    0.0   50.0
 2002-10-06    0.0   51.0   52.0   79.0    0.0   49.0
 2002-10-05    0.0   54.0   56.0   69.0    0.0   46.0,
 Timestamp('2002-11-24 00:00:00'):             zone0  zone1  zone2  zone3  zone4  zone5
 2002-11-24    0.0   33.0   47.0   65.0    0.0   44.0
 2002-11-23    0.0   54.0   39.0   75.0    0.0   44.0
 2002-11-22    0.0   50.0   52.0   78.0    0.0   53.0
 2002-11-21    0.0   47.0   53.0   70.0    0.0   41.0
 2002-11-20    0.0   51.0   50.0   82.0    0.0   40.0
 2002-11-19  